In [ ]:
import planet
import os
import json
import requests
from requests.auth import HTTPBasicAuth
import asyncio

from pathlib import Path
root = Path.cwd()

os.environ['PL_API_KEY']='PLAK50ae7fa8d1ef48dc82d762d8b29745c2'
PL_API_KEY = os.getenv('PL_API_KEY')



In [73]:
PL_API_KEY = os.getenv('PL_API_KEY')

In [5]:
import geopandas as gpd
import pandas as pd

site_folder = 'HARV'
trees_df = pd.read_csv(root / 'output' / site_folder / f'neon_trees_{site_folder}.csv')
geometry = gpd.points_from_xy(trees_df.easting_tree, trees_df.northing_tree, crs="EPSG:32618")
trees_df['geometry'] = geometry
trees_gdf = gpd.GeoDataFrame(trees_df,geometry='geometry',crs='EPSG:32618')

In [12]:
trees_gdf.columns

Index(['Unnamed: 0', 'X', 'plotID', 'individualID', 'easting_tree',
       'northing_tree', 'taxonID', 'height', 'ninetyCrownDiameter',
       'canopyPosition', 'adjCoordinateUncertainty', 'utmZone', 'easting_plot',
       'northing_plot', 'chm_height', 'chm_dif', 'geometry'],
      dtype='object')

In [23]:
import folium

map_coords = tuple(trees_gdf.to_crs(4326).get_coordinates().iloc[1550,]) # get xy coords for index 37
map_coords = tuple([map_coords[1],map_coords[0]]) 

buff = trees_gdf.iloc[1281:1283,].buffer(600)

m = folium.Map(location=map_coords, zoom_start=11)

folium.GeoJson(
    trees_gdf.to_crs(4326),
    marker=folium.Circle(radius=2, fill_color="orange", fill_opacity=0.4, color="black", weight=1),
    tooltip=folium.GeoJsonTooltip(fields=["Unnamed: 0","taxonID"])
).add_to(m)

folium.GeoJson(
    buff.to_crs(4326).envelope
).add_to(m)

m

In [46]:
import numpy as np

y = buff.to_crs(4326).envelope.get_coordinates().iloc[0:5:,0]
x = buff.to_crs(4326).envelope.get_coordinates().iloc[0:5:,1]

xy_coords = [[np.round(i[1],6),np.round(i[0],6)] for i in zip(x,y)]

xy_coords

[[np.float64(-72.185166), np.float64(42.531733)],
 [np.float64(-72.170571), np.float64(42.531733)],
 [np.float64(-72.170571), np.float64(42.542526)],
 [np.float64(-72.185166), np.float64(42.542526)],
 [np.float64(-72.185166), np.float64(42.531733)]]

In [53]:
geojson_geometry = {
  "type": "Polygon",
  "coordinates": [xy_coords]
}
geojson_geometry

{'type': 'Polygon',
 'coordinates': [[[np.float64(-72.185166), np.float64(42.531733)],
   [np.float64(-72.170571), np.float64(42.531733)],
   [np.float64(-72.170571), np.float64(42.542526)],
   [np.float64(-72.185166), np.float64(42.542526)],
   [np.float64(-72.185166), np.float64(42.531733)]]]}

In [76]:
# define parameters of post request

# geojson bounding box
date_filter = {
    "type": "DateRangeFilter", 
    "field_name": "acquired", # The field to filter on
    "config": {
        "gte": "2019-01-01T00:00:00.000Z",
         "lte": "2019-12-31T00:00:00.000Z" 
    }
}

geometry_filter = {
    "type":"GeometryFilter",
    "field_name": "geometry",
    "config": geojson_geometry
}

instrument_filter = {
    "type": "StringInFilter",
    "field_name": "instrument",
    "config": ["PS2"]
}

cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}


combined_filter = {
    "type": "AndFilter",
    "config": [instrument_filter, geometry_filter, date_filter,cloud_cover_filter]
}

# Construct the request.
search_request = {
    "item_types" : ["PSScene"],
    "filter" : combined_filter
}

In [77]:
# find item ids by querying Data API

search_result = requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PL_API_KEY, ''),
    json=search_request)

search_result

results = search_result.json()
image_ids = [feature['id'] for feature in results['features']]
print(image_ids)

['20190318_151233_103e', '20190313_151504_100c', '20190306_151536_1012', '20190305_151350_101f', '20190301_151235_1003', '20191205_151635_1034', '20191205_151633_1034', '20191224_133543_0f49', '20191223_133608_1048', '20191223_133607_1048', '20191222_133557_0f3c', '20191222_133556_1_0f3c', '20191221_133515_1052', '20191220_151617_1001', '20191219_133504_100d', '20191219_133504_1_100d', '20191218_133619_104b', '20191207_134144_0f46', '20191130_151950_1005', '20191126_151918_1005', '20191126_152054_103d', '20191126_152055_103d', '20191121_134228_0f32', '20191121_134227_1_0f32', '20191117_134149_1048', '20191116_134432_0f44', '20191116_134433_0f44', '20191115_151853_1040', '20191115_151854_1040', '20191113_134313_1043', '20191109_151603_1010', '20191108_134422_0f36', '20191102_134409_1052', '20191102_151708_1039', '20191102_151707_1039', '20191101_134357_0f46', '20191101_134358_0f46', '20191024_134607_0f33', '20191024_134608_0f33', '20191024_151727_100c', '20191024_151726_100c', '20191021

In [78]:
results

{'_links': {'_first': 'https://api.planet.com/data/v1/searches/0b55b55c8a4d4e8fb25f38960b3a6177/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D',
  '_next': 'https://api.planet.com/data/v1/searches/0b55b55c8a4d4e8fb25f38960b3a6177/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIxLTAzLTAxVDIzOjU2OjQ2LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTkwMTAzXzE1MTIwOV8xMDQyIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319',
  '_self': 'https://api.planet.com/data/v1/searches/0b55b55c8a4d4e8fb25f38960b3a6177/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D'},
 'features': [{'_links': {'_self': 'https://ap

In [70]:
# build order using item ids; define aoi to clip to
def create_request(name,aoi,items):
    # In practice, you will use a Data API search to find items
        order_details = planet.order_request.build_request(
        name=name,
        products=[
            planet.order_request.product(item_ids=items,
                                            product_bundle='analytic_sr_udm2',
                                            item_type='PSScene')
        ],
        tools=[planet.order_request.clip_tool(aoi=aoi)])

        return order_details

async def create_and_download(client, order_detail, directory):
   with planet.reporting.StateBar(state='creating') as reporter:
       order = await client.create_order(order_detail)
       reporter.update(state='created', order_id=order['id'])
       await client.wait(order['id'], callback=reporter.update_state)

   await client.download_order(order['id'], directory, progress_bar=True)

DOWNLOAD_DIR = root / 'planet_data' 



In [79]:
async def main():
   async with planet.Session() as sess:
       cl = sess.client('orders')

       # Create the order request
       request = create_request('harv_order',geojson_geometry,image_ids)

       # Create and download the order
       order = await create_and_download(cl, request, DOWNLOAD_DIR)

In [80]:
await main()

00:01 - order  - state: creating


BadQuery: {"field":{"Details":[{"message":"no access to assets: PSScene/20191005_152354_1105/[ortho_analytic_4b_sr]"},{"message":"no access to assets: PSScene/20190929_180635_0f4c/[ortho_analytic_4b_sr ortho_analytic_4b_xml]"},{"message":"no access to assets: PSScene/20190929_180634_0f4c/[ortho_analytic_4b_sr ortho_analytic_4b_xml]"},{"message":"no access to assets: PSScene/20190817_151826_0f4e/[ortho_analytic_4b_sr ortho_analytic_4b_xml]"},{"message":"no access to assets: PSScene/20190816_152141_1105/[ortho_analytic_4b_sr]"},{"message":"no access to assets: PSScene/20190706_151934_1105/[ortho_analytic_4b_sr]"},{"message":"no access to assets: PSScene/20190702_192407_0f1c/[ortho_analytic_4b_sr ortho_analytic_4b_xml]"},{"message":"no access to assets: PSScene/20190702_192408_1_0f1c/[ortho_analytic_4b_sr ortho_analytic_4b_xml]"},{"message":"no access to assets: PSScene/20190609_180522_0f02/[ortho_analytic_4b_sr ortho_analytic_4b_xml]"},{"message":"no access to assets: PSScene/20190905_18